## ARIMA Models

ARIMA stands for Autoregressive Integrated Moving Average. Compared with the above model it uses a linear combination of past time steps, and moving averages to predict t.


#### ARIMA Models Evaluated
1. Multi-setp output ARIMA






ARIMA takes only a stationary time series. As explored in the ***DATA ANALYSIS*** notebook the load data can be made stationary by ***ANALYSIS RESULTS***



We will use the ARIMA model from statsmodels.api which takes the following arguments:
- p: is the number of lag observations in he model. Lag order.
- d: the number of times raw observations are differenced. Degree of differencing.
- q: the side of the moving average window. The order of moving average.


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('dark')

from statsmodels.tsa.arima_model import ARIMA

In [5]:
from model_persistence import get_persistence_dataset, train_test_split, walk_forward_evaluation, calculate_errors, plot_error

In [26]:
data = get_persistence_dataset()
train, test = train_test_split(data)
train.head(3)

,t-0 h_0,t-0 h_1,t-0 h_2,t-0 h_3,t-0 h_4,t-0 h_5,t-0 h_6,t-0 h_7,t-0 h_8,t-0 h_9,...,t-0 h_14,t-0 h_15,t-0 h_16,t-0 h_17,t-0 h_18,t-0 h_19,t-0 h_20,t-0 h_21,t-0 h_22,t-0 h_23
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01,25385.0,24382.0,22734.0,21286.0,20264.0,19905.0,20010.0,20377.0,20094.0,20637.0,...,24672.0,23528.0,23118.0,23606.0,26447.0,28020.0,29014.0,29571.0,29031.0,26798.0
2015-01-02,27070.0,24935.0,23214.0,22540.0,22096.0,22066.0,22275.0,23025.0,23699.0,25834.0,...,29241.0,27973.0,27320.0,27157.0,29045.0,31967.0,33266.0,33668.0,32426.0,29705.0
2015-01-03,25564.0,23584.0,22194.0,21296.0,21145.0,21316.0,21674.0,22127.0,22756.0,24654.0,...,26416.0,25362.0,24770.0,24501.0,24992.0,27964.0,29694.0,29874.0,28979.0,26863.0


In [27]:
### reshape the data (flatten) for input into ARIMA

train.values[:2]

array([[25385., 24382., 22734., 21286., 20264., 19905., 20010., 20377.,
        20094., 20637., 22250., 23547., 24133., 24713., 24672., 23528.,
        23118., 23606., 26447., 28020., 29014., 29571., 29031., 26798.],
       [27070., 24935., 23214., 22540., 22096., 22066., 22275., 23025.,
        23699., 25834., 27927., 29072., 29071., 29439., 29241., 27973.,
        27320., 27157., 29045., 31967., 33266., 33668., 32426., 29705.]])

In [30]:
train = train.values.flatten()

In [29]:
data.values.flatten()

array([25385., 24382., 22734., ..., 28071., 25801., 24455.])

In [ ]:
test = ARIMA(train, order=(24, 0, 0))
    
test.fit()
    
    
test.predict(len(data.values.flatten()),len(data.values.flatten())+23)